## CIFAR10 Hparam search

In [2]:
import torch
import numpy as np
import os, sys
import shutil
import pickle
import cv2
from tqdm import tqdm
from sklearn.cluster import KMeans
from datetime import datetime

sys.path.insert(0, 'src')
from utils.utils import ensure_dir, read_json, informal_log
from utils.visualizations import plot
from utils.model_utils import prepare_device

import model.metric as module_metric
import model.loss as module_loss
import datasets.datasets as module_data
import model.model as module_model

from src.train import main as train_fn
from predict import predict
from parse_config import ConfigParser


sys.path.insert(0, 'setup')
from setup_cifar10 import setup_cifar10 
# import cv2
# print(cv2.__version__)

In [3]:
config_path = 'configs/train_cifar10_sift_explainer.json'
debug = False
if debug:
    learning_rates = [1e-6] #, 1e-5, 1e-4, 1e-3, 5e-2, 1e-2, 5e-1, 1e-1]
    weight_decays = [0, 1e-1] #, 1e-2, 1e-3]
else:
    learning_rates = [1e-4, 1e-3, 5e-2, 1e-2, 5e-1, 1e-1]
    weight_decays = [0, 1e-1, 1e-2, 1e-3]

config_json = read_json(config_path)


### Set up data loaders

In [4]:
dataset_args = config_json['dataset']['args']
train_descriptors_dataset = module_data.KDDataset(split='train', **dataset_args)
test_descriptors_dataset = module_data.KDDataset(split='test', **dataset_args)

dataloader_args = config_json['data_loader']['args']
train_descriptors_dataloader = torch.utils.data.DataLoader(
    train_descriptors_dataset,
    shuffle=True,
    **dataloader_args)
test_descriptors_dataloader = torch.utils.data.DataLoader(
    test_descriptors_dataset,
    shuffle=False,
    **dataloader_args)

device, device_ids = prepare_device(config_json['n_gpu'])
metric_fns = [getattr(module_metric, met) for met in config_json['metrics']]
loss_fn = getattr(module_loss, config_json['loss'])

### Run Hyperparameter search

In [5]:
best = {
    'lr': -1,
    'wd': -1,
    'val_acc': -1
}
n_trials = len(learning_rates) * len(weight_decays)
trial_idx = 1
timestamp = datetime.now().strftime(r'%m%d_%H%M%S')

# Logging
log_path = os.path.join(config_json['trainer']['save_dir'], timestamp, 'log.txt')
ensure_dir(os.path.dirname(log_path))
informal_log("Hyperparameter search", log_path)
informal_log("Learning rates: {}".format(learning_rates), log_path)
informal_log("Weight decays: {}".format(weight_decays), log_path)

# 

# Debug mode
if debug:
    config_json['trainer']['epochs'] = 1
    
for lr in learning_rates:
    for wd in weight_decays:
        # Update config json
        config_json['optimizer']['args'].update({
            'lr': lr,
            'weight_decay': wd
        })
        
        # Create run ID for trial
        itr_timestamp = datetime.now().strftime(r'%m%d_%H%M%S')
        informal_log("[{}] Trial {}/{}: LR = {} WD = {}".format(
            itr_timestamp, trial_idx, n_trials, lr, wd), log_path)
        run_id = os.path.join(timestamp, 'trials', 'lr_{}-wd_{}'.format(lr, wd))
        config = ConfigParser(config_json, run_id=run_id)
        print(config.config['optimizer']['args'])
        
        # Train model
        model = train_fn(
            config=config, 
            train_data_loader=train_descriptors_dataloader,
            val_data_loader=test_descriptors_dataloader)
        
        # Restore model
        model_restore_path = os.path.join(config.save_dir, 'model_best.pth')
        
        model.restore_model(model_restore_path)
        print("restored model")
        # Run on validation set using predict function
        
        trial_path = os.path.dirname(os.path.dirname(model_restore_path))
        output_save_path = os.path.join(trial_path, "val_outputs.pth")
        log_save_path = os.path.join(trial_path, "val_metrics.pth")
        
        validation_data = predict(
            data_loader=test_descriptors_dataloader,
            model=model,
            metric_fns=metric_fns,
            device=device,
            loss_fn=loss_fn,
            output_save_path=output_save_path,
            log_save_path=log_save_path)
       
        # Obtain accuracy and compare to previous best
        print(validation_data['metrics'].keys())
        val_accuracy = validation_data['metrics']['accuracy']
        if val_accuracy > best['val_acc']:
            best.update({
                'lr': lr,
                'wd': wd,
                'val_acc': val_accuracy
            })
            informal_log("Best accuracy of {:.3f} with lr={} and wd={}".format(val_accuracy, lr, wd), log_path)
            informal_log("Trial path: {}".format(trial_path), log_path)
            # Copy model and outputs to 1 directory for easy access
            best_save_dir = os.path.join(os.path.dirname(os.path.dirname(trial_path)), 'best')
            ensure_dir(best_save_dir)
            best_outputs_save_path = os.path.join(best_save_dir, 'outputs.pth')
            best_model_save_path = os.path.join(best_save_dir, 'model.pth')
            torch.save(validation_data['logits'], best_outputs_save_path)
            model.save_model(best_model_save_path)
            informal_log("Saved model and outputs to {}".format(best_save_dir), log_path)
            
            
        trial_idx += 1

Hyperparameter search
Learning rates: [0.0001, 0.001, 0.05, 0.01, 0.5, 0.1]
Weight decays: [0, 0.1, 0.01, 0.001]
[0605_194638] Trial 1/24: LR = 0.0001 WD = 0
OrderedDict([('lr', 0.0001), ('weight_decay', 0)])
Created LinearLayers model with 20010 trainable parameters
Training from scratch.
Checkpoint save directory: saved/cifar10/resnet18/explainer/sift_descriptor_histogram/cifar32_32_sigma1.6/dense_stride_4/2000means/hard_labels/0605_194638/trials/lr_0.0001-wd_0/models
Train Epoch: 1 [0/196 (0%)] Loss: 2.302115
Train Epoch: 1 [48/196 (24%)] Loss: 2.301745
Train Epoch: 1 [96/196 (49%)] Loss: 2.301603
Train Epoch: 1 [144/196 (73%)] Loss: 2.302063
Train Epoch: 1 [192/196 (98%)] Loss: 2.302346
    epoch          : 1
    val_TP         : [ 21   0   0 126  70   0 834   0   0   0]
    val_TN         : [8918 9000 9000 7949 8391 9000 1793 9000 9000 9000]
    val_FPs        : [  82    0    0 1051  609    0 7207    0    0    0]
    val_FNs        : [ 979 1000 1000  874  930 1000  166 1000 1000 1

/n/fs/ac-alignment/explain-alignment/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-alignment/explain-alignment/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


Train Epoch: 2 [0/196 (0%)] Loss: 2.303248
Train Epoch: 2 [48/196 (24%)] Loss: 2.301825
Train Epoch: 2 [96/196 (49%)] Loss: 2.302167
Train Epoch: 2 [144/196 (73%)] Loss: 2.302609
Train Epoch: 2 [192/196 (98%)] Loss: 2.302439
    epoch          : 2
    val_TP         : [ 21   0   0 128  70   0 832   0   0   0]
    val_TN         : [8916 9000 9000 7946 8389 9000 1800 9000 9000 9000]
    val_FPs        : [  84    0    0 1054  611    0 7200    0    0    0]
    val_FNs        : [ 979 1000 1000  872  930 1000  168 1000 1000 1000]
    val_accuracy   : 0.1051
    val_per_class_accuracy: [0.8937 0.9    0.9    0.8074 0.8459 0.9    0.2632 0.9    0.9    0.9   ]
    val_per_class_accuracy_mean: 0.8210200000000001
    val_precision  : [0.2        0.         0.         0.10829103 0.10279001 0.
 0.10358566 0.         0.         0.        ]
    val_precision_mean: 0.051466670420370585
    val_recall     : [0.021 0.    0.    0.128 0.07  0.    0.832 0.    0.    0.   ]
    val_recall_mean: 0.1051
    val_

/n/fs/ac-alignment/explain-alignment/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-alignment/explain-alignment/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


Train Epoch: 3 [0/196 (0%)] Loss: 2.302732
Train Epoch: 3 [48/196 (24%)] Loss: 2.302791
Train Epoch: 3 [96/196 (49%)] Loss: 2.302033
Train Epoch: 3 [144/196 (73%)] Loss: 2.302895
Train Epoch: 3 [192/196 (98%)] Loss: 2.302477
    epoch          : 3
    val_TP         : [ 21   0   0 128  70   0 831   0   0   0]
    val_TN         : [8914 9000 9000 7945 8385 9000 1806 9000 9000 9000]
    val_FPs        : [  86    0    0 1055  615    0 7194    0    0    0]
    val_FNs        : [ 979 1000 1000  872  930 1000  169 1000 1000 1000]
    val_accuracy   : 0.105
    val_per_class_accuracy: [0.8935 0.9    0.9    0.8073 0.8455 0.9    0.2637 0.9    0.9    0.9   ]
    val_per_class_accuracy_mean: 0.8210000000000001
    val_precision  : [0.19626168 0.         0.         0.10819949 0.10218978 0.
 0.1035514  0.         0.         0.        ]
    val_precision_mean: 0.051020235794892475
    val_recall     : [0.021 0.    0.    0.128 0.07  0.    0.831 0.    0.    0.   ]
    val_recall_mean: 0.10500000000000

...

Train Epoch: 57 [0/196 (0%)] Loss: 2.285451
Train Epoch: 57 [48/196 (24%)] Loss: 2.284225
Train Epoch: 57 [96/196 (49%)] Loss: 2.282512
Train Epoch: 57 [144/196 (73%)] Loss: 2.287645
Train Epoch: 57 [192/196 (98%)] Loss: 2.288355
    epoch          : 57
    val_TP         : [505 586 230  36 239 591  13 111 419 151]
    val_TN         : [7570 7583 8333 8863 8237 7260 8985 8939 8288 8823]
    val_FPs        : [1430 1417  667  137  763 1740   15   61  712  177]
    val_FNs        : [495 414 770 964 761 409 987 889 581 849]
    val_accuracy   : 0.2881
    val_per_class_accuracy: [0.8075 0.8169 0.8563 0.8899 0.8476 0.7851 0.8998 0.905  0.8707 0.8974]
    val_per_class_accuracy_mean: 0.85762
    val_precision  : [0.26098191 0.29256116 0.25641026 0.20809249 0.23852295 0.25353925
 0.46428571 0.64534884 0.37046861 0.46036585]
    val_precision_mean: 0.3450577036999244
    val_recall     : [0.505 0.586 0.23  0.036 0.239 0.591 0.013 0.111 0.419 0.151]
    val_recall_mean: 0.28809999999999997
    

100%|███████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 48.53it/s]


Saving validation results to saved/cifar10/resnet18/explainer/sift_descriptor_histogram/cifar32_32_sigma1.6/dense_stride_4/2000means/hard_labels/0605_194638/trials/lr_0.1-wd_0.001
restored model


100%|███████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 63.73it/s]

dict_keys(['TP', 'TN', 'FPs', 'FNs', 'accuracy', 'per_class_accuracy', 'per_class_accuracy_mean', 'precision', 'precision_mean', 'recall', 'recall_mean', 'predicted_class_distribution', 'f1', 'f1_mean'])


### Check best model's results on training data

In [14]:
# Trained on soft labels with 75 clusters
# model_restore_path = 'saved/cifar10/resnet18/explainer/sift_descriptor_histogram/hparam_search/0525_103130/best/model.pth'
# Trained on hard GT labels with 75 clusters
model_restore_path = 'saved/cifar10/resnet18/explainer/sift_descriptor_histogram/true_labels/75_clusters/models/model_best.pth'
# Variable
restore_dir = 'saved/cifar10/resnet18/explainer/sift_descriptor_histogram/cifar32_32/soft_labels/0529_130252'
model_restore_path = os.path.join(restore_dir, 'models/model_best.pth')
config_json = read_json(os.path.join(restore_dir, 'models/config.json'))

device, device_ids = prepare_device(config_json['n_gpu'])
metric_fns = [getattr(module_metric, met) for met in config_json['metrics']]
loss_fn = getattr(module_loss, config_json['loss'])

model_args = config_json['arch']['args']
model = module_model.LinearLayers(
    checkpoint_path=model_restore_path,
    **model_args)

print(model)

model.eval()
model = model.to(device)

# Dataloaders
dataset_args = config_json['dataset']['args']
train_descriptors_dataset = module_data.KDDataset(split='train', **dataset_args)
test_descriptors_dataset = module_data.KDDataset(split='test', **dataset_args)

dataloader_args = config_json['data_loader']['args']
train_descriptors_dataloader = torch.utils.data.DataLoader(
    train_descriptors_dataset,
    shuffle=True,
    **dataloader_args)
test_descriptors_dataloader = torch.utils.data.DataLoader(
    test_descriptors_dataset,
    shuffle=False,
    **dataloader_args)



# Evaluate on validation set as a sanity check
validation_results = predict(
        data_loader=test_descriptors_dataloader,
        model=model,
        metric_fns=metric_fns,
        device=device,
        loss_fn=loss_fn,
        output_save_path=None,
        log_save_path=None)

print("Validation accuracy: {}".format(validation_results['metrics']['accuracy']))

training_results = predict(
        data_loader=train_descriptors_dataloader,
        model=model,
        metric_fns=metric_fns,
        device=device,
        loss_fn=loss_fn,
        output_save_path=None,
        log_save_path=None)

print("Training accuracy: {}".format(training_results['metrics']['accuracy']))

LinearLayers(
  (layers): Sequential(
    (0): Linear(in_features=75, out_features=10, bias=True)
  )
)
Trainable parameters: 760


100%|███████████████████████████████████████████| 40/40 [00:00<00:00, 97.39it/s]


Validation accuracy: 0.2411


100%|████████████████████████████████████████| 196/196 [00:00<00:00, 326.32it/s]

Training accuracy: 0.24014
